In [1]:
# pip install pymorphy2 >> None

In [1]:
import pandas as pd
import os
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# задаем директорию
file_path = 'C:/Users/Пользователь/Desktop/Hack_LinkedIn'

In [4]:
# открываем файлы с данными
#posts = pd.read_csv(f'{file_path}/posts.csv')
#profiles = pd.read_csv(f'{file_path}/profiles.csv')
posts = pd.read_csv('posts.csv')
profiles = pd.read_csv('profiles.csv')

In [5]:
# открываем датафрейм с постами
posts.head(2)

,Unnamed: 0,user_id,text,likes,comments,reposts
0,0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0
1,1,ali-wodan,I’m #hiring. Know anyone who might be interested?,1,0,0


In [6]:
# открываем датафрейм с информацией о пользователях
profiles.head(2)

,Unnamed: 0,id,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
0,0,ali-wodan,Ali Wodan,Head of Design,Performix,Head Of Design,"podcast, it","Москва, Московская область, Россия",2 391,500+,"I am a digital product design lead. I've been designing digital systems for years. Teams creating, management, design process building, user research, analytics etc. it's all that i like to do. Also I am into psychology, music, philosophy."
1,1,ikotow,Игорь Котов,Директор по производству – Технократия,Технократия,Технократия,"it, обучение, менеджмент, технологии, производство","Казань, Республика Татарстан, Россия",340,338,Making IT production great again! Создатель сервиса для управления ресурсами: Resourcer.appМой telegram-канал: https://t.me/pastikotow


In [7]:
# переименуем столбец text в post для лучшего отражения содержимого
posts = posts.rename(columns={'text': 'post'})

**Предобработка данных**

In [8]:
# функция удаления эмодзи
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # смайлики
                               u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                               u"\U0001F680-\U0001F6FF"  # транспорт и символы на карте
                               u"\U0001F1E0-\U0001F1FF"  # флаги
                               u"\U00002500-\U00002BEF"  # китайские символы
                               # другие разные символы
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642" 
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Удаляем эмодзи, используя паттерны
    text_without_emojis = emoji_pattern.sub(r'', text)
    return text_without_emojis

# удаляем эмодзи из постов
posts['post'] = posts['post'].apply(lambda x: remove_emojis(x) if pd.notnull(x) else x)

In [9]:
# функция лемматизации текста
morph = pymorphy2.MorphAnalyzer()
def lemmatize_text(text):
    lemmatized_words = [morph.parse(word)[0].normal_form for word in text.split()]
    return ' '.join(lemmatized_words)

# лемматизируем посты
posts['post_lemmatized'] = posts['post'].apply(lemmatize_text)

In [10]:
# удаляем лишние текстовые символы (те, которые не состоят из букв русского алфавита)
posts['post_lemmatized'] = posts['post_lemmatized'].str.replace('[^а-яА-ЯёЁ\s]', '', regex=True) # только русские буквы и пробелы

# Шаблон '\W' в методе str.replace() представляет любой символ, не состоящий из слов, в т.ч. пробелы.
# Шаблон '[^\w\s]' пробелы оставляет, '[^\w\s\d]' + цифры

In [12]:
# скачиваем стоп-слова 
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krasn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# еще один список от bukvarix.com - список стоп-слов Яндекс Wordstat - (этот список можно дополнить/изменить)
current_directory = os.getcwd()
#file_path_words = f'{file_path}\stop_words.txt'
file_path_words = f'stop_words.txt'
with open(file_path_words, 'r', encoding='utf-8') as file:
    stop_words_buk = file.read()

In [14]:
# удаляем стоп-слова и слова-паразиты
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words_buk]))

In [15]:
# определяем шаблон регулярного выражения для русских букв
pattern = '[^а-яА-ЯёЁ]'

# создаем маску, чтобы проверить, содержит ли каждая ячейка русские буквы
mask = posts['post_lemmatized'].str.contains(pattern, regex=True)

# фильтруем датафрейм, используя маску
posts = posts[mask]

In [16]:
posts.sample(20)

,Unnamed: 0,user_id,post,likes,comments,reposts,post_lemmatized
738,738,melekes,Заметка о безопасности,0,0,0,заметка безопасность
3074,3074,votumam,Жду ваши итоги года в своей ленте,6,2,1,ждать ваш итог год лента
1373,1373,%D0%B0%D0%BD%D0%B3%D0%B5%D0%BB%D0%B8%D0%BD%D0%B0-%D0%BE%D0%BD%D0%B4%D0%B0%D1%80-834b41227,"Под напором родителей, заполнила резюме по специальности:( но меня очень тянет дизайнить. Я все таки надеюсь что смогу устроиться дизайнером в классную компанию.День 8 по поиску работы, отправляю резюме и пишу сопроводительные письма. Выполнила 2 тестовых задания пока ответа нет, но я на этом не останавливаюсь, изучаю разные компании и стартапы. Я очень рада за ребят, которые уже устроились на работу они мотивируют меня не сдаваться и я на правильном пути. #дизайнер #вакансии #opentowork #designer #uxuidesigner",132,11,3,напор родителей заполнить резюме специальности очень тянуть дизайнить всё надеяться смочь устроиться дизайнер классный компаниюдеть поиск работы отправлять резюме писать сопроводительный письма выполнить тестовый задание пока ответ останавливаюсь изучать разный компания стартапы очень рада ребят устроиться работа мотивировать сдаваться правильный пути дизайнер вакансия
3102,3102,elizaveta-kalinina-3aab58214,"#неделя_2 На этот плакат меня вдохновил Маяковский) Я придумала, что это будет плакат для поэтического вечера на тему футуризма. Без Маяковского я просто не могу представить себе футуризм, поэтому я показала его в громком образе и как главное действующее лицо.Слова буквально вылетают из его рта и кричат на зрителя. Я использовала известную цитату из его произведения. Что чувствуете от этого плаката?) #poster #graphicdesign #design",4,0,0,неделя плакат вдохновить маяковский придумала плакат поэтический вечер тема футуризма маяковский просто представить футуризм поэтому показать громкий образ главное действовать лицослово буквально вылетать рот кричать зрителя использовать известный цитата произведения чувствовать плаката
4678,4678,oleh-frolov-webnauts,"(English below)Завжди приємно на vue.js щось дуже швидке і зручне, а потім – розповісти про це у LinkedIn. Тож – тримайте кейс про розробку адмінки для керування ігровою платформою від наших фронтенщиків. Крута візуалізація даних, експорт, імпорт, взаємодія із беком з питання розпізнавання аудіо та інші фічі – у наявності. Детальніше можна почитати тут: https://lnkd.in/ensD56hzIt is always a pleasure to develop something really fast and convenient on vue.js and share it on LinkedIn. Here you go. Game platform control admin panel development case all the way from our frontenders. Cool data visualization, export, import, interaction with backend (audio recognition and other features) aboard. Details on the link.#vuejs #cases #development",4,0,0,завждать примный щося дуже швидка зручне потм розповсть це тож тримайт кейс розробка адмнка керуванний гровой платформа вд фронтенщикв крутой взуалзацть даних експорт мпорт взамодть бек питанний розпзнаванний аудый нш фч наявност детальнш можный почитать
868,868,designeragaltsova,"Здравствуйте! Если вы только начали отслеживать меня, то вот о чем я пишу: #design, #game, #3d и #illustration.Сообщите мне, какая тема вас интересует больше всего!",0,0,0,здравствуйте начать отслеживать пишу сообщить тема интересовать большой
830,830,ordalion,"Тестовые собеседования! Всем привет! Все не по наслышке знают, о волнение и некотором дискомфорте перед собеседованием. Мы с командой хотим узнать, что вы думаете о тестовых собеседованиях, где ментор будет иметь возможность подсказать необходимый вектор развития :) Буду, очень благодарен, если поможете распространить Пройдите, пожалуйста, опрос:https://lnkd.in/dbsWu3KM #cv #тестовоесобеседование #собес #скриниг",23,1,3,тестовый собеседования привет всё наслышко знают волнение некоторый дискомфорт собеседованием команда хотеть узнать думать тестовый собеседованиях ментор иметь возможность подсказать необходимый вектор развитие очень благодарен помочь распространить пройдите пожал

In [17]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1848 entries, 0 to 5054
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       1848 non-null   int64 
 1   user_id          1848 non-null   object
 2   post             1848 non-null   object
 3   likes            1848 non-null   object
 4   comments         1848 non-null   int64 
 5   reposts          1848 non-null   int64 
 6   post_lemmatized  1848 non-null   object
dtypes: int64(3), object(4)
memory usage: 115.5+ KB


Объединяем датафреймы

In [18]:
# переименуем столбец id в user_id в датафрейме profiles, для последующего объединения с posts
profiles = profiles.rename(columns={'id': 'user_id'})

In [19]:
# объединяем датафреймы
df = pd.merge(posts, profiles, on='user_id')

In [20]:
# удаляем лишние столбцы
df = df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)

In [21]:
# удаляем дубликаты
df.drop_duplicates(inplace=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1841 entries, 0 to 1847
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           1841 non-null   object
 1   post              1841 non-null   object
 2   likes             1841 non-null   object
 3   comments          1841 non-null   int64 
 4   reposts           1841 non-null   int64 
 5   post_lemmatized   1841 non-null   object
 6   user_name         1841 non-null   object
 7   user_head         1841 non-null   object
 8   user_work         1760 non-null   object
 9   user_position     1841 non-null   object
 10  user_tags         477 non-null    object
 11  user_location     1834 non-null   object
 12  user_viewers      1841 non-null   object
 13  user_contacts     1841 non-null   object
 14  user_common_info  1688 non-null   object
dtypes: int64(2), object(13)
memory usage: 230.1+ KB


In [23]:
df.sample(5)

,user_id,post,likes,comments,reposts,post_lemmatized,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
1,ali-wodan,"Хэй честной народ! Ищу векторного иллюстратора на проект, с возможным длительным сотрудничеством по итогу. Можно удаленно. Уровень иллюстраций не хуже чем в примере https://lnkd.in/gkrvhxYРепост, пожалуйста)Если ты умеешь так рисовать, напиши мне в личку :-)Hey! I am looking for an Illustrator who can help us with a project! You need to have skills like in the example below or upper. Text me to discuss prices and due :-)#вакансия #vacancy #illustration #иллюстрация #project #проект",6,0,0,хэй честной народ искать векторный иллюстратор проект возможный длительный сотрудничество итогу удаленно уровень иллюстрация плохой пример репост пожалуйстаести уметь рисовать написать личка вакансия иллюстрация проект,Ali Wodan,Head of Design,Performix,Head Of Design,"podcast, it","Москва, Московская область, Россия",2 391,500+,"I am a digital product design lead. I've been designing digital systems for years. Teams creating, management, design process building, user research, analytics etc. it's all that i like to do. Also I am into psychology, music, philosophy."
197,olgaperun,"Друзья! На прошлой недели «Погнали в IT» стартовал! .Первое онлайн-занятие ———-Расскажу поподробнее, чтобы было понятно, как и что у нас организовано.Студент выбрал направление «Управление проектами»..На первой бесплатной консультации мы проговорили цель и задачи его обучения. Он планирует изменить сферу деятельности и уйти в IT целиком, начиная с нуля. По окончанию обучения (с учетом самостоятельного развития в том числе) он должен быть готовым занять позицию Junior в IT-компании..На ознакомительной встрече мы также обсудили стоимость и удобный график оплаты. На основании потребности Студента я составила план 25 занятий (темы, тезисы, основной материал, домашние задания и др.).На первом занятии мы разобрали базисные проектные понятия и их взаимодействие. Именно первое занятие вводит в профессию, знакомит с лексикой и терминологией проектной деятельности, на нем я объясняю структурные моменты. ... У моего Студента глаза загорелись, жажда новых знаний – это замечательный стимул к развитию! Домашнее задание состоит из 2 частей, самостоятельно нужно сделать 2, но я как эксперт всегда на связи со студентом.Следующая онлайн-встреча через 3 дня.Мысль дня! Проекты - это главная движущая сила изменений!Мое впечатление: Быть тьютором – необыкновенно ответственно и непередаваемо драйвово! Ощущение, что я открываю для другого человека новый неизведанный мир, полный классных открытий!#погналивit #клубэкспертов #клубэкспертовit #этопросто #обучаемснулявit #управлениепроектами #системнаяаналитика #devops #frontdelivery #backdelivery",2,0,0,друзья прошлый неделя погнать стартовал первое онлайнзанятие рассказать поподробнее понятно организованостудент выбрать направление управление проектамиич первый бесплатный консультация проговорить цель задача обучения планировать изменить сфера деятельность уйти целиком начинать нуля окончание обучение учёт самостоятельный развитие числе должный готовый занять позиция компанииич ознакомительный встреча также обсудить стоимость удобный график оплаты основание потребность студент составить план занятие темы тезисы основной материал домашний задание дрич первый занятие разобрать базисный проектный понятие взаимодействие именно первый занятие вводить профессию знакомить лексика терминология проектный деятельности немой объяснять структурный моменты студент глаз загорелись жажда новый знание замечательный стимул развитию домашний задание состоять частей самостоятельно нужно сделать эксперт связь студентомследовать онлайнвстреча днямысль дня проект главный движущий сила изменениймый впечатление тьютор необыкновенно ответственно непередаваемый драйвово ощущение открывать человек новый неизведанный мир полный классный открытийпогналив клубэксперт клубэкспертов этопросто обучае

In [26]:
# сохраняем датасет
df.to_csv('linkedin.csv')